Check azure-ai-ml

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.29.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: c:\Users\Tony\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Connect to my workspace

In [2]:
#import    required    libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
#Enter    details    of    your    Azure    Machine    Learning    workspace
subscription_id = 'c6cdfd9c-3767-4be7-a343-0acd27ddefb9'
resource_group = 'rg-tshen-0917'
workspace = 'ml-ws-tshen-0917'
#connect to the workspace
ml_client    =    MLClient(DefaultAzureCredential(),    subscription_id,
resource_group,    workspace)
ml_client

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000002547BEB1FD0>,
         subscription_id=c6cdfd9c-3767-4be7-a343-0acd27ddefb9,
         resource_group_name=rg-tshen-0917,
         workspace_name=ml-ws-tshen-0917)

## Use the Python SDK to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [15]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
parser = argparse.ArgumentParser()
parser.add_argument("--data", type=str, help="Path to input data")
parser.add_argument("--test_size", type=float, default=0.30, help="Proportion of the dataset to include in the test split")
parser.add_argument("--reg", type=float, default=0.01, help="Regularization rate parameter")
args = parser.parse_args()
args

print("Loading Data...")
diabetes = pd.read_csv(args.data)

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=args.test_size)

# set regularization hyperparameter
reg = args.reg

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg, 'test size of', args.test_size)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

Overwriting src/diabetes-training.py


Run diabetes-training.py remotely in ML Workspace on ami-cluster


In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-pythonv2-train",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Run diabetes-training.py locally

In [16]:
!python ./src/diabetes-training.py --data ./data/diabetes.csv --test_size 0.20 --reg 0.02

Loading Data...
Training a logistic regression model with regularization rate of 0.02 test size of 0.2
Accuracy: 0.777
AUC: 0.8571124495721323
